In [410]:
import pandas as pd
import numpy as np


In [411]:
### 인허가 데이터 하나로 합치기 ###
baths = pd.read_csv('../data/processed/인허가/인허가_목욕장업_일자별_물재생센터_영업현황_20170101_20250630.csv', encoding='utf-8-sig')
laundry = pd.read_csv('../data/processed/인허가/인허가_세탁업_일자별_물재생센터_영업현황_20170101_20250630.csv', encoding='utf-8-sig')
pool = pd.read_csv('../data/processed/인허가/인허가_수영장업_일자별_물재생센터_영업현황_20170101_20250630.csv', encoding='utf-8-sig')
sports = pd.read_csv('../data/processed/인허가/인허가_종합체육시설업_일자별_물재생센터_영업현황_20170101_20250630.csv', encoding='utf-8-sig')
gym = pd.read_csv('../data/processed/인허가/인허가_체력단련장업_일자별_물재생센터_영업현황_20170101_20250630.csv', encoding='utf-8-sig')

print(baths.columns, baths.shape)
print(baths.head())

print(laundry.columns, laundry.shape)
print(laundry.head())

print(pool.columns, pool.shape)
print(pool.head())

print(sports.columns, sports.shape)
print(sports.head())

print(gym.columns, gym.shape)
print(gym.head())

Index(['date', '물재생센터', '업종', '시설수_운영'], dtype='object') (12412, 4)
         date    물재생센터    업종  시설수_운영
0  2017-01-01  난지물재생센터  목욕장업     210
1  2017-01-01  서남물재생센터  목욕장업     455
2  2017-01-01  중랑물재생센터  목욕장업     353
3  2017-01-01  탄천물재생센터  목욕장업     189
4  2017-01-02  난지물재생센터  목욕장업     210
Index(['date', '물재생센터', '업종', '시설수_운영'], dtype='object') (12412, 4)
         date    물재생센터   업종  시설수_운영
0  2017-01-01  난지물재생센터  세탁업     939
1  2017-01-01  서남물재생센터  세탁업    2253
2  2017-01-01  중랑물재생센터  세탁업    2117
3  2017-01-01  탄천물재생센터  세탁업    1018
4  2017-01-02  난지물재생센터  세탁업     939
Index(['date', '물재생센터', '업종', '시설수_운영'], dtype='object') (12412, 4)
         date    물재생센터    업종  시설수_운영
0  2017-01-01  난지물재생센터  수영장업      28
1  2017-01-01  서남물재생센터  수영장업      64
2  2017-01-01  중랑물재생센터  수영장업      46
3  2017-01-01  탄천물재생센터  수영장업      39
4  2017-01-02  난지물재생센터  수영장업      28
Index(['date', '물재생센터', '업종', '시설수_운영'], dtype='object') (12412, 4)
         date    물재생센터       업종  시설수_운영
0  2017-01-01  난지물재생센터  종합체육

In [412]:
# 인허가 데이터 하나로 합치기
license = pd.concat([baths, laundry, pool, sports, gym], ignore_index=True)
license

,date,물재생센터,업종,시설수_운영
0,2017-01-01,난지물재생센터,목욕장업,210
1,2017-01-01,서남물재생센터,목욕장업,455
2,2017-01-01,중랑물재생센터,목욕장업,353
3,2017-01-01,탄천물재생센터,목욕장업,189
4,2017-01-02,난지물재생센터,목욕장업,210
...,...,...,...,...
62055,2025-06-29,탄천물재생센터,체력단련장업,894
62056,2025-06-30,난지물재생센터,체력단련장업,618
62057,2025-06-30,서남물재생센터,체력단련장업,1737
62058,2025-06-30,중랑물재생센터,체력단련장업,1071


In [413]:
# 피벗: (date, 물재생센터, 업종) -> 시설수_운영
pivot_license = license.pivot_table(
    index="date", 
    columns=["물재생센터", "업종"], 
    values="시설수_운영"
)

# 컬럼명 평탄화: "난지_목욕장업" 형식으로 (물재생센터 제거)
pivot_license.columns = [f"{center.replace('물재생센터','')}_{biz}" for center, biz in pivot_license.columns]

pivot_license = pivot_license.fillna(0).astype(int)

# 날짜 컬럼 복구
pivot_license = pivot_license.reset_index().rename(columns={"date": "날짜"})

print(pivot_license.shape)
pivot_license.head()

(3103, 21)


,날짜,난지_목욕장업,난지_세탁업,난지_수영장업,난지_종합체육시설업,난지_체력단련장업,서남_목욕장업,서남_세탁업,서남_수영장업,서남_종합체육시설업,...,중랑_목욕장업,중랑_세탁업,중랑_수영장업,중랑_종합체육시설업,중랑_체력단련장업,탄천_목욕장업,탄천_세탁업,탄천_수영장업,탄천_종합체육시설업,탄천_체력단련장업
0,2017-01-01,210,939,28,26,779,455,2253,64,48,...,353,2117,46,35,1355,189,1018,39,41,1144
1,2017-01-02,210,939,28,26,779,455,2253,64,48,...,353,2116,46,35,1355,189,1017,39,41,1143
2,2017-01-03,210,939,28,26,779,454,2253,64,48,...,353,2113,46,35,1355,189,1017,39,41,1143
3,2017-01-04,210,939,28,26,779,454,2253,64,48,...,353,2113,46,35,1355,189,1017,39,41,1143
4,2017-01-05,210,939,28,26,779,454,2253,64,48,...,353,2113,46,35,1355,189,1017,39,41,1143


In [414]:
pivot_license.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3103 entries, 0 to 3102
Data columns (total 21 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   날짜          3103 non-null   object
 1   난지_목욕장업     3103 non-null   int64 
 2   난지_세탁업      3103 non-null   int64 
 3   난지_수영장업     3103 non-null   int64 
 4   난지_종합체육시설업  3103 non-null   int64 
 5   난지_체력단련장업   3103 non-null   int64 
 6   서남_목욕장업     3103 non-null   int64 
 7   서남_세탁업      3103 non-null   int64 
 8   서남_수영장업     3103 non-null   int64 
 9   서남_종합체육시설업  3103 non-null   int64 
 10  서남_체력단련장업   3103 non-null   int64 
 11  중랑_목욕장업     3103 non-null   int64 
 12  중랑_세탁업      3103 non-null   int64 
 13  중랑_수영장업     3103 non-null   int64 
 14  중랑_종합체육시설업  3103 non-null   int64 
 15  중랑_체력단련장업   3103 non-null   int64 
 16  탄천_목욕장업     3103 non-null   int64 
 17  탄천_세탁업      3103 non-null   int64 
 18  탄천_수영장업     3103 non-null   int64 
 19  탄천_종합체육시설업  3103 non-null   int64 
 20  탄천_체력단련장

In [415]:
### 일자별로 정리되어있는 다른 데이터들 ###
climate = pd.read_csv('../data/processed/climate_add_DI.csv', encoding='utf-8-sig')
wastewater = pd.read_csv('../data/processed/하수처리량_센터별처리장별_일자기준_총합.csv', encoding='utf-8-sig')
population = pd.read_csv('../data/processed/인구/처리센터별_일평균생활인구_201701_202506.csv', encoding='utf-8-sig') ### 인구 데이터 오류 해결한 새로운 데이터로 변경
date = pd.read_csv('../data/raw/results/전체날짜_공휴일포함_2017_2025.csv', encoding='utf-8-sig')
river = pd.read_csv('../data/processed/하수/센터_하천_일평균강수.csv', encoding='utf-8-sig')

In [416]:
wastewater = wastewater.rename(columns={'측정일자' : '날짜'})
wastewater

,날짜,난지_1처리장,난지_2처리장,난지_정화조,난지_중계펌프장,서남_1처리장,서남_2처리장,서남_시설현대화,중랑_1처리장,중랑_2처리장,중랑_3처리장,중랑_4처리장,탄천_1처리장,탄천_2처리장,탄천_합계,중랑_합계,서남_합계,난지_합계,총처리량
0,2017-01-01,212474.0,272541.0,725.0,7134,506592,909476,0.0,178815,196443.0,665615,173440,340895.0,285320,626215.0,1214313.0,1416068.0,492874.0,3749470.0
1,2017-01-02,224083.0,286338.0,4147.0,5968,530211,925685,0.0,187500,223691.0,680506,145380,364085.0,311049,675134.0,1237077.0,1455896.0,520536.0,3888643.0
2,2017-01-03,205711.0,269835.0,4554.0,8354,525809,940373,0.0,183910,220646.0,692097,127880,373454.0,301380,674834.0,1224533.0,1466182.0,488454.0,3854003.0
3,2017-01-04,219468.0,277305.0,5100.0,8307,524292,936281,0.0,179602,227335.0,678904,139240,368112.0,302954,671066.0,1225081.0,1460573.0,510180.0,3866900.0
4,2017-01-05,207189.0,275215.0,5733.0,8152,523867,931948,0.0,64000,236299.0,696193,219140,357531.0,298534,656065.0,1215632.0,1455815.0,496289.0,3823801.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3098,2025-06-26,0.0,0.0,0.0,0,0,0,0.0,204960,259968.0,665020,88690,410644.0,325456,736100.0,1218638.0,0.0,0.0,1954738.0
3099,2025-06-27,0.0,0.0,0.0,0,0,0,0.0,211670,272448.0,651760,77990,388450.0,332700,721150.0,1213868.0,0.0,0.0,1935018.0
3100,2025-06-28,0.0,0.0,0.0,0,0,0,0.0,211400,271488.0,671460,78480,395823.0,321781,717604.0,1232828.0,0.0,0.0,1950432.0
3101,2025-06-29,0.0,0.0,0.0,0,0,0,0.0,210730,277824.0,627580,90120,421497.0,300016,721513.0,1206254.0,0.0,0.0,1927767.0


In [417]:
population.columns

Index(['날짜', '난지물재생센터', '서남물재생센터', '중랑물재생센터', '탄천물재생센터'], dtype='object')

In [418]:
population= population.rename(columns={
    '난지물재생센터' : '난지_생활인구', 
    '서남물재생센터' : '서남_생활인구', 
    '중랑물재생센터' : '중랑_생활인구', 
    '탄천물재생센터' : '탄천_생활인구'
})

# 숫자형 컬럼 전체 소수 둘째 자리까지 반올림
population.iloc[:, 1:] = population.iloc[:, 1:].round(2)

population

,날짜,난지_생활인구,서남_생활인구,중랑_생활인구,탄천_생활인구
0,2017-01-01,1920764.58,3894314.30,3591517.68,1860066.54
1,2017-01-02,2011902.49,4161642.56,3615305.84,1990395.60
2,2017-01-03,2021411.02,4163121.84,3631853.44,2008282.48
3,2017-01-04,2016146.44,4147625.47,3639234.09,1999705.18
4,2017-01-05,2019225.84,4136014.04,3638478.08,1996612.67
...,...,...,...,...,...
3098,2025-06-26,1820483.24,3761501.63,3208494.51,2015511.30
3099,2025-06-27,1818477.77,3749692.87,3200508.38,2008208.52
3100,2025-06-28,1751408.17,3585215.01,3186658.49,1948102.33
3101,2025-06-29,1716949.92,3556494.17,3193967.83,1924061.00


In [419]:
date

,날짜,요일,공휴일
0,2017-01-01,일요일,신정
1,2017-01-02,월요일,NaN
2,2017-01-03,화요일,NaN
3,2017-01-04,수요일,NaN
4,2017-01-05,목요일,NaN
...,...,...,...
3100,2025-06-26,목요일,NaN
3101,2025-06-27,금요일,NaN
3102,2025-06-28,토요일,NaN
3103,2025-06-29,일요일,NaN


In [420]:
# 날짜 형식으로 변환
date['날짜'] = pd.to_datetime(date['날짜'])

# 날짜 범위 생성 
expected_dates = pd.date_range(start='2017-01-01', end='2025-06-30')

# 누락된 날짜 확인
missing_dates = expected_dates.difference(date['날짜'])

# 중복된 날짜 확인
duplicate_dates = date['날짜'][date['날짜'].duplicated()]

# 총 일자 확인
actual_days = date['날짜'].nunique()
expected_days = len(expected_dates)

print(f"총 기대 일수: {expected_days}일")
print(f"실제 고유 일수: {actual_days}일")

if len(missing_dates) == 0:
    print("모든 날짜가 포함되어 있습니다.")
else:
    print("누락된 날짜가 있습니다:")
    print(missing_dates)

if len(duplicate_dates) == 0:
    print("중복된 날짜는 없습니다.")
else:
    print("중복된 날짜가 있습니다:")
    print(duplicate_dates)

총 기대 일수: 3103일
실제 고유 일수: 3103일
모든 날짜가 포함되어 있습니다.
중복된 날짜가 있습니다:
276    2017-10-03
3048   2025-05-05
Name: 날짜, dtype: datetime64[ns]


In [421]:
date[date['날짜']=='2017-10-03']

,날짜,요일,공휴일
275,2017-10-03,화요일,개천절
276,2017-10-03,화요일,추석


In [422]:
# 공휴일 여부를 0/1로 변환
date['공휴일'] = date['공휴일'].notna().astype(int)
date

,날짜,요일,공휴일
0,2017-01-01,일요일,1
1,2017-01-02,월요일,0
2,2017-01-03,화요일,0
3,2017-01-04,수요일,0
4,2017-01-05,목요일,0
...,...,...,...
3100,2025-06-26,목요일,0
3101,2025-06-27,금요일,0
3102,2025-06-28,토요일,0
3103,2025-06-29,일요일,0


In [423]:
# 총 3101행이어야하는데 3105행이 있음 중복 확인 후 제거
# 중복 제거 (첫 번째 행만 남김)
date = date.drop_duplicates(subset='날짜', keep='first').reset_index(drop=True)

# 중복 제거 후 총 일자 확인
print(f"중복 제거 후 일자 수: {len(date)}")

중복 제거 후 일자 수: 3103


In [424]:
date[date['날짜']=='2025-05-05']

,날짜,요일,공휴일
3046,2025-05-05,월요일,1


In [425]:
river.columns

Index(['날짜', '난지_구파발천', '난지_녹번천', '난지_봉원천', '난지_불광천', '난지_창릉천', '난지_향동천',
       '난지_홍제천', '서남_대방천', '서남_도림천', '서남_도림천지류2', '서남_목감천', '서남_반포천', '서남_봉천천',
       '서남_사당천', '서남_시흥천', '서남_안양천', '서남_오류천', '중랑_가오천', '중랑_당현천', '중랑_대동천',
       '중랑_면목천', '중랑_묵동천', '중랑_방학천', '중랑_성북천', '중랑_우이천', '중랑_월곡천', '중랑_전농천',
       '중랑_정릉천', '중랑_중랑천', '중랑_청계천', '중랑_화계천', '탄천_감이천', '탄천_고덕천', '탄천_대사골천',
       '탄천_망월천', '탄천_성내천', '탄천_세곡천', '탄천_양재천', '탄천_여의천', '탄천_탄천'],
      dtype='object')

In [426]:
river

,날짜,난지_구파발천,난지_녹번천,난지_봉원천,난지_불광천,난지_창릉천,난지_향동천,난지_홍제천,서남_대방천,서남_도림천,...,중랑_화계천,탄천_감이천,탄천_고덕천,탄천_대사골천,탄천_망월천,탄천_성내천,탄천_세곡천,탄천_양재천,탄천_여의천,탄천_탄천
0,2017-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00
1,2017-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00
2,2017-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00
3,2017-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00
4,2017-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3098,2025-06-26,1.5,1.5,0.5,1.0,1.5,0.5,0.5,0.5,0.5,...,1.0,0.5,0.5,0.5,0.5,0.5,0.75,0.75,0.5,0.75
3099,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00
3100,2025-06-28,1.0,1.0,0.0,0.5,1.0,0.0,0.0,0.5,0.5,...,1.0,0.0,0.5,0.5,0.5,0.0,0.00,0.00,0.0,0.00
3101,2025-06-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00


In [427]:
### river의 하천 컬럼들을 센터/하천명으로 분해해 MultiIndex로 만들고, 컬럼의 상위레벨(센터) 기준으로 합계를 구해 '센터_하천' 컬럼 생성 ###

river_tmp = river.copy()

# 숫자형 하천 컬럼만 분리 (날짜 제외)
num_cols = [c for c in river_tmp.columns if c != "날짜"]

# "센터_하천명"을 ('센터','하천명')의 튜플로 분해하여 MultiIndex 생성
#  예: "탄천_감이천" -> ("탄천","감이천")
col_tuples = [tuple(c.split("_", 1)) for c in num_cols]  # 첫 '_'만 분리
multi_cols = pd.MultiIndex.from_tuples(col_tuples, names=["센터", "하천"])

# 분해한 MultiIndex를 컬럼에 부착
river_tmp_num = river_tmp[num_cols]
river_tmp_num.columns = multi_cols

# 센터 레벨(상위레벨)로 가로방향 합계 NaN은 자동으로 무시되어 합함
center_sum = river_tmp_num.groupby(level="센터", axis=1).sum()

# 최종 형식으로 정리: '날짜' + '센터_하천' 형식의 컬럼명
river_sum = pd.concat([river_tmp[["날짜"]], center_sum], axis=1)
river_sum.iloc[:, 1:] = river_sum.iloc[:, 1:].round(2) # 소수점 2째자리까지만
river_sum.columns = ["날짜"] + [f"{c}_하천" for c in center_sum.columns]

# 결과 확인
print(river_sum.shape)
print(river_sum.head())


(3103, 5)
           날짜  난지_하천  서남_하천  중랑_하천  탄천_하천
0  2017-01-01    0.0   0.00   0.00    0.0
1  2017-01-02    0.0   0.67   2.25    0.0
2  2017-01-03    0.0   0.00   0.00    0.0
3  2017-01-04    0.0   0.00   0.00    0.0
4  2017-01-05    0.0   0.00   0.00    0.0


C:\Users\user\AppData\Local\Temp\ipykernel_21996\3271371327.py:18: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  center_sum = river_tmp_num.groupby(level="센터", axis=1).sum()


## '날짜' 컬럼 기준 1차 데이터 병합 - 공휴일, 인허가, 인구, 하천 

In [428]:
# 날짜 컬럼 모두 datetime 형식으로 변환
pivot_license['날짜'] = pd.to_datetime(pivot_license['날짜'])
date['날짜'] = pd.to_datetime(date['날짜'])
population['날짜'] = pd.to_datetime(population['날짜'])
wastewater['날짜'] = pd.to_datetime(wastewater['날짜'])
river_sum['날짜'] = pd.to_datetime(river_sum['날짜'])

In [429]:
### 공휴일 + 인허가 ###
total = pd.merge(date, pivot_license, on = '날짜', how = 'left')
total

,날짜,요일,공휴일,난지_목욕장업,난지_세탁업,난지_수영장업,난지_종합체육시설업,난지_체력단련장업,서남_목욕장업,서남_세탁업,...,중랑_목욕장업,중랑_세탁업,중랑_수영장업,중랑_종합체육시설업,중랑_체력단련장업,탄천_목욕장업,탄천_세탁업,탄천_수영장업,탄천_종합체육시설업,탄천_체력단련장업
0,2017-01-01,일요일,1,210,939,28,26,779,455,2253,...,353,2117,46,35,1355,189,1018,39,41,1144
1,2017-01-02,월요일,0,210,939,28,26,779,455,2253,...,353,2116,46,35,1355,189,1017,39,41,1143
2,2017-01-03,화요일,0,210,939,28,26,779,454,2253,...,353,2113,46,35,1355,189,1017,39,41,1143
3,2017-01-04,수요일,0,210,939,28,26,779,454,2253,...,353,2113,46,35,1355,189,1017,39,41,1143
4,2017-01-05,목요일,0,210,939,28,26,779,454,2253,...,353,2113,46,35,1355,189,1017,39,41,1143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3098,2025-06-26,목요일,0,120,515,22,20,618,238,1247,...,212,1120,38,31,1071,99,561,34,29,895
3099,2025-06-27,금요일,0,120,515,22,20,618,238,1247,...,212,1120,38,31,1071,99,561,34,29,894
3100,2025-06-28,토요일,0,120,515,22,20,618,238,1247,...,212,1120,38,31,1071,99,561,34,29,894
3101,2025-06-29,일요일,0,120,515,22,20,618,238,1247,...,212,1120,38,31,1071,99,561,34,29,894


In [430]:
### (공휴일+인허가) + 인구 ###
total = pd.merge(total, population, on = '날짜', how = 'left')
total

,날짜,요일,공휴일,난지_목욕장업,난지_세탁업,난지_수영장업,난지_종합체육시설업,난지_체력단련장업,서남_목욕장업,서남_세탁업,...,중랑_체력단련장업,탄천_목욕장업,탄천_세탁업,탄천_수영장업,탄천_종합체육시설업,탄천_체력단련장업,난지_생활인구,서남_생활인구,중랑_생활인구,탄천_생활인구
0,2017-01-01,일요일,1,210,939,28,26,779,455,2253,...,1355,189,1018,39,41,1144,1920764.58,3894314.30,3591517.68,1860066.54
1,2017-01-02,월요일,0,210,939,28,26,779,455,2253,...,1355,189,1017,39,41,1143,2011902.49,4161642.56,3615305.84,1990395.60
2,2017-01-03,화요일,0,210,939,28,26,779,454,2253,...,1355,189,1017,39,41,1143,2021411.02,4163121.84,3631853.44,2008282.48
3,2017-01-04,수요일,0,210,939,28,26,779,454,2253,...,1355,189,1017,39,41,1143,2016146.44,4147625.47,3639234.09,1999705.18
4,2017-01-05,목요일,0,210,939,28,26,779,454,2253,...,1355,189,1017,39,41,1143,2019225.84,4136014.04,3638478.08,1996612.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3098,2025-06-26,목요일,0,120,515,22,20,618,238,1247,...,1071,99,561,34,29,895,1820483.24,3761501.63,3208494.51,2015511.30
3099,2025-06-27,금요일,0,120,515,22,20,618,238,1247,...,1071,99,561,34,29,894,1818477.77,3749692.87,3200508.38,2008208.52
3100,2025-06-28,토요일,0,120,515,22,20,618,238,1247,...,1071,99,561,34,29,894,1751408.17,3585215.01,3186658.49,1948102.33
3101,2025-06-29,일요일,0,120,515,22,20,618,238,1247,...,1071,99,561,34,29,894,1716949.92,3556494.17,3193967.83,1924061.00


In [431]:
### (공휴일+인허가 + 인구) + 하수처리량 ###
total = pd.merge(total, wastewater, on = '날짜', how = 'left')
total

,날짜,요일,공휴일,난지_목욕장업,난지_세탁업,난지_수영장업,난지_종합체육시설업,난지_체력단련장업,서남_목욕장업,서남_세탁업,...,중랑_2처리장,중랑_3처리장,중랑_4처리장,탄천_1처리장,탄천_2처리장,탄천_합계,중랑_합계,서남_합계,난지_합계,총처리량
0,2017-01-01,일요일,1,210,939,28,26,779,455,2253,...,196443.0,665615,173440,340895.0,285320,626215.0,1214313.0,1416068.0,492874.0,3749470.0
1,2017-01-02,월요일,0,210,939,28,26,779,455,2253,...,223691.0,680506,145380,364085.0,311049,675134.0,1237077.0,1455896.0,520536.0,3888643.0
2,2017-01-03,화요일,0,210,939,28,26,779,454,2253,...,220646.0,692097,127880,373454.0,301380,674834.0,1224533.0,1466182.0,488454.0,3854003.0
3,2017-01-04,수요일,0,210,939,28,26,779,454,2253,...,227335.0,678904,139240,368112.0,302954,671066.0,1225081.0,1460573.0,510180.0,3866900.0
4,2017-01-05,목요일,0,210,939,28,26,779,454,2253,...,236299.0,696193,219140,357531.0,298534,656065.0,1215632.0,1455815.0,496289.0,3823801.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3098,2025-06-26,목요일,0,120,515,22,20,618,238,1247,...,259968.0,665020,88690,410644.0,325456,736100.0,1218638.0,0.0,0.0,1954738.0
3099,2025-06-27,금요일,0,120,515,22,20,618,238,1247,...,272448.0,651760,77990,388450.0,332700,721150.0,1213868.0,0.0,0.0,1935018.0
3100,2025-06-28,토요일,0,120,515,22,20,618,238,1247,...,271488.0,671460,78480,395823.0,321781,717604.0,1232828.0,0.0,0.0,1950432.0
3101,2025-06-29,일요일,0,120,515,22,20,618,238,1247,...,277824.0,627580,90120,421497.0,300016,721513.0,1206254.0,0.0,0.0,1927767.0


In [432]:
### (공휴일+인허가 + 인구 + 하수처리량) + 하천 ###
total = pd.merge(total,  river_sum, on = '날짜', how = 'left')
total

,날짜,요일,공휴일,난지_목욕장업,난지_세탁업,난지_수영장업,난지_종합체육시설업,난지_체력단련장업,서남_목욕장업,서남_세탁업,...,탄천_2처리장,탄천_합계,중랑_합계,서남_합계,난지_합계,총처리량,난지_하천,서남_하천,중랑_하천,탄천_하천
0,2017-01-01,일요일,1,210,939,28,26,779,455,2253,...,285320,626215.0,1214313.0,1416068.0,492874.0,3749470.0,0.0,0.00,0.00,0.00
1,2017-01-02,월요일,0,210,939,28,26,779,455,2253,...,311049,675134.0,1237077.0,1455896.0,520536.0,3888643.0,0.0,0.67,2.25,0.00
2,2017-01-03,화요일,0,210,939,28,26,779,454,2253,...,301380,674834.0,1224533.0,1466182.0,488454.0,3854003.0,0.0,0.00,0.00,0.00
3,2017-01-04,수요일,0,210,939,28,26,779,454,2253,...,302954,671066.0,1225081.0,1460573.0,510180.0,3866900.0,0.0,0.00,0.00,0.00
4,2017-01-05,목요일,0,210,939,28,26,779,454,2253,...,298534,656065.0,1215632.0,1455815.0,496289.0,3823801.0,0.0,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3098,2025-06-26,목요일,0,120,515,22,20,618,238,1247,...,325456,736100.0,1218638.0,0.0,0.0,1954738.0,7.0,6.00,8.25,5.25
3099,2025-06-27,금요일,0,120,515,22,20,618,238,1247,...,332700,721150.0,1213868.0,0.0,0.0,1935018.0,0.0,0.00,0.00,0.00
3100,2025-06-28,토요일,0,120,515,22,20,618,238,1247,...,321781,717604.0,1232828.0,0.0,0.0,1950432.0,3.5,4.33,12.00,1.50
3101,2025-06-29,일요일,0,120,515,22,20,618,238,1247,...,300016,721513.0,1206254.0,0.0,0.0,1927767.0,0.0,0.00,0.00,0.00


## 기후 데이터도 합치기

In [433]:
climate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83607 entries, 0 to 83606
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   날짜             83607 non-null  object 
 1   지점             83607 non-null  int64  
 2   지점명            83607 non-null  object 
 3   평균습도(%)        83607 non-null  float64
 4   최저습도(%)        83607 non-null  float64
 5   최고습도(%)        83607 non-null  float64
 6   습도표준편차         83607 non-null  float64
 7   일_평균기온(°C)     83607 non-null  float64
 8   일_최저기온(°C)     83607 non-null  float64
 9   일_최고기온(°C)     83607 non-null  float64
 10  일_일강수량(mm)     83607 non-null  float64
 11  일_최대순간풍속(m/s)  83606 non-null  float64
 12  일_평균풍속(m/s)    83607 non-null  float64
 13  주소             83607 non-null  object 
 14  위도             83607 non-null  float64
 15  경도             83607 non-null  float64
 16  행정구            83607 non-null  object 
 17  불쾌지수(DI)       83607 non-null  float64
 18  불쾌지수등급

In [434]:
climate.columns

Index(['날짜', '지점', '지점명', '평균습도(%)', '최저습도(%)', '최고습도(%)', '습도표준편차',
       '일_평균기온(°C)', '일_최저기온(°C)', '일_최고기온(°C)', '일_일강수량(mm)', '일_최대순간풍속(m/s)',
       '일_평균풍속(m/s)', '주소', '위도', '경도', '행정구', '불쾌지수(DI)', '불쾌지수등급'],
      dtype='object')

In [435]:
climate['불쾌지수등급']

0           쾌적
1           쾌적
2           쾌적
3           쾌적
4           쾌적
         ...  
83602    약간 불쾌
83603    약간 불쾌
83604       불쾌
83605       불쾌
83606       불쾌
Name: 불쾌지수등급, Length: 83607, dtype: object

In [436]:
### 필요한 기후 데이터만 필터링, 불쾌지수등급은 object 타입이니까 일단 제거하고 나중에 센터별로 만들어지면 해당 값에 대해 쓰기 ###
# 사용할 컬럼 :  '날짜', '평균습도(%)', '최저습도(%)', '최고습도(%)', '습도표준편차', '일_평균기온(°C)', '일_최저기온(°C)', '일_최고기온(°C)', '일_일강수량(mm)', '일_최대순간풍속(m/s)', '일_평균풍속(m/s)', '행정구', '불쾌지수(DI)', '불쾌지수등급'
# 같은 날짜에 대해 행정구가 겹친다면 평균값 사용 -> 하나의 행정구에 대해 여러 관측 값이 있어도 평균 사용할거라 ㄱㅊ

# 사용할 컬럼
use_columns = [
    '날짜', '행정구',
    '평균습도(%)', '최저습도(%)', '최고습도(%)', '습도표준편차',
    '일_평균기온(°C)', '일_최저기온(°C)', '일_최고기온(°C)',
    '일_일강수량(mm)', '일_최대순간풍속(m/s)', '일_평균풍속(m/s)',
    '불쾌지수(DI)'
]

# 필요한 컬럼만 추출
climate_filtered = climate[use_columns]

# 날짜 + 행정구 단위로 평균값
climate_grouped = climate_filtered.groupby(['날짜', '행정구'], as_index=False).mean()

climate_grouped


,날짜,행정구,평균습도(%),최저습도(%),최고습도(%),습도표준편차,일_평균기온(°C),일_최저기온(°C),일_최고기온(°C),일_일강수량(mm),일_최대순간풍속(m/s),일_평균풍속(m/s),불쾌지수(DI)
0,2017-01-01,강남구,77.375000,66.0,89.0,7.906038,3.50,-0.40,7.00,0.0,3.30,1.30,40.751419
1,2017-01-01,강동구,83.416667,71.0,94.0,8.203481,2.30,-1.90,6.40,0.0,3.20,0.90,38.133814
2,2017-01-01,강북구,78.875000,69.0,89.0,7.097535,3.80,-0.30,7.50,0.0,3.50,0.70,41.066153
3,2017-01-01,강서구,68.208333,60.0,78.0,5.356663,4.80,1.10,7.80,0.0,3.90,1.70,43.675468
4,2017-01-01,관악구,85.708333,68.0,95.0,9.679558,3.55,-0.95,7.65,0.0,5.00,1.35,39.931428
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74304,2025-06-30,영등포구,72.062500,57.0,87.0,10.316960,28.70,25.60,32.70,0.0,6.25,1.60,79.717449
74305,2025-06-30,용산구,70.500000,58.0,83.0,8.732747,28.60,25.70,32.10,0.0,8.90,2.70,79.345870
74306,2025-06-30,은평구,75.500000,60.0,91.0,10.073642,26.90,22.30,31.10,0.0,6.50,1.00,77.398905
74307,2025-06-30,중구,80.500000,61.0,96.0,11.037486,26.80,23.50,31.70,0.0,4.90,0.60,77.854760


In [437]:
### 강수량 정보는 나중에 행정동 면적 계산해서 쓸거니까 따로 뺴기
precipitation = climate_grouped.copy()
precipitation = precipitation[['날짜', '행정구', '일_일강수량(mm)']]
precipitation

,날짜,행정구,일_일강수량(mm)
0,2017-01-01,강남구,0.0
1,2017-01-01,강동구,0.0
2,2017-01-01,강북구,0.0
3,2017-01-01,강서구,0.0
4,2017-01-01,관악구,0.0
...,...,...,...
74304,2025-06-30,영등포구,0.0
74305,2025-06-30,용산구,0.0
74306,2025-06-30,은평구,0.0
74307,2025-06-30,중구,0.0


In [438]:
precipitation.to_csv('../data/processed/강수량.csv', encoding='utf-8-sig', index=False)

In [439]:
center = pd.read_excel('../data/raw/seoul_mapping_center_to_districts.xlsx', engine='openpyxl')
center

,물재생센터,관할지역,전체_일부
0,중랑물재생센터,동대문구,전역
1,중랑물재생센터,중랑구,전역
2,중랑물재생센터,성북구,전역
3,중랑물재생센터,노원구,전역
4,중랑물재생센터,강북구,전역
5,중랑물재생센터,도봉구,전역
6,중랑물재생센터,광진구,전역
7,중랑물재생센터,종로구,일부
8,중랑물재생센터,중구,일부
9,중랑물재생센터,성동구,일부


In [440]:
center = center.rename(columns={'관할지역': '행정구'})
center

,물재생센터,행정구,전체_일부
0,중랑물재생센터,동대문구,전역
1,중랑물재생센터,중랑구,전역
2,중랑물재생센터,성북구,전역
3,중랑물재생센터,노원구,전역
4,중랑물재생센터,강북구,전역
5,중랑물재생센터,도봉구,전역
6,중랑물재생센터,광진구,전역
7,중랑물재생센터,종로구,일부
8,중랑물재생센터,중구,일부
9,중랑물재생센터,성동구,일부


In [441]:
# climate_grouped랑 합치기
climate_with_center = pd.merge(
    climate_grouped, center,
    on='행정구',
    how='left'  # → 기후 데이터에 없는 행정구는 NaN으로 남기기
)

In [442]:
climate_with_center.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86672 entries, 0 to 86671
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   날짜             86672 non-null  object 
 1   행정구            86672 non-null  object 
 2   평균습도(%)        86672 non-null  float64
 3   최저습도(%)        86672 non-null  float64
 4   최고습도(%)        86672 non-null  float64
 5   습도표준편차         86672 non-null  float64
 6   일_평균기온(°C)     86672 non-null  float64
 7   일_최저기온(°C)     86672 non-null  float64
 8   일_최고기온(°C)     86672 non-null  float64
 9   일_일강수량(mm)     86672 non-null  float64
 10  일_최대순간풍속(m/s)  86671 non-null  float64
 11  일_평균풍속(m/s)    86672 non-null  float64
 12  불쾌지수(DI)       86672 non-null  float64
 13  물재생센터          86672 non-null  object 
 14  전체_일부          86672 non-null  object 
dtypes: float64(11), object(4)
memory usage: 9.9+ MB


In [443]:
climate_with_center = climate_with_center.replace({
    '물재생센터': {
        '탄천물재생센터': '탄천',
        '중랑물재생센터': '중랑',
        '난지물재생센터': '난지',
        '서남물재생센터': '서남'
    }
})

In [444]:
# 필요한 컬럼만 추출(강수량은 제외 : 행정동별 면적 비율로 넣어줄거니까)
df = climate_with_center[ [
    '날짜', '물재생센터',
    '평균습도(%)', '최저습도(%)', '최고습도(%)', '습도표준편차',
    '일_평균기온(°C)', '일_최저기온(°C)', '일_최고기온(°C)',
    '일_최대순간풍속(m/s)', '일_평균풍속(m/s)', '불쾌지수(DI)'
]]

# 그룹바이: 날짜 + 물재생센터 기준으로 평균
grouped = df.groupby(['날짜', '물재생센터']).mean().reset_index()

# 피벗: 물재생센터별로 컬럼 펼치기. 날짜를 기준으로 행(row)을 만들고 물재생센터별로 **기후 지표가 열(column)**이 되도록 펼쳐줌
pivoted = grouped.pivot(index='날짜', columns='물재생센터')

# 컬럼 정리: 다중 인덱스를 단일 이름으로
pivoted.columns = [
    f"{center}_{col}" for col, center in pivoted.columns
]

# 인덱스 리셋
pivoted = pivoted.reset_index()

# 소수점 둘째 자리까지 반올림
pivoted.iloc[:, 1:] = pivoted.iloc[:, 1:].round(2)

pivoted


,날짜,난지_평균습도(%),서남_평균습도(%),중랑_평균습도(%),탄천_평균습도(%),난지_최저습도(%),서남_최저습도(%),중랑_최저습도(%),탄천_최저습도(%),난지_최고습도(%),...,중랑_일_최대순간풍속(m/s),탄천_일_최대순간풍속(m/s),난지_일_평균풍속(m/s),서남_일_평균풍속(m/s),중랑_일_평균풍속(m/s),탄천_일_평균풍속(m/s),난지_불쾌지수(DI),서남_불쾌지수(DI),중랑_불쾌지수(DI),탄천_불쾌지수(DI)
0,2017-01-01,77.11,78.60,80.54,78.89,65.67,65.78,68.67,67.25,87.17,...,3.48,3.22,1.13,0.99,0.74,1.05,41.04,41.44,39.91,40.41
1,2017-01-02,77.49,77.28,79.31,77.73,61.67,61.50,65.11,61.75,89.00,...,6.99,5.25,1.65,1.53,1.47,1.42,44.26,44.93,44.02,44.85
2,2017-01-03,63.13,62.31,67.32,61.32,42.17,43.72,43.33,40.75,83.00,...,6.37,5.62,1.43,1.23,1.39,1.30,40.93,41.77,40.76,42.53
3,2017-01-04,60.61,61.59,63.29,60.07,32.00,28.94,36.44,32.00,81.67,...,6.23,5.40,1.27,1.20,1.20,1.18,43.66,44.23,43.70,44.77
4,2017-01-05,55.00,54.83,59.72,56.81,43.83,45.28,50.11,45.25,63.17,...,7.40,5.90,2.43,1.68,2.06,1.45,44.70,45.15,43.93,44.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3098,2025-06-26,88.15,84.70,85.41,84.18,77.17,70.22,73.78,72.75,96.83,...,4.91,4.50,1.02,1.16,0.99,1.08,69.53,70.51,70.56,70.87
3099,2025-06-27,79.09,75.54,79.29,76.58,61.50,58.61,62.44,59.75,91.83,...,4.46,5.03,0.97,0.97,0.93,1.08,73.74,74.61,74.05,74.62
3100,2025-06-28,85.97,81.58,84.40,80.02,79.00,73.44,76.00,69.75,93.00,...,4.87,5.30,0.90,1.11,0.81,1.02,76.13,77.13,76.46,77.47
3101,2025-06-29,79.58,76.18,75.91,72.94,62.50,58.33,60.56,58.50,92.50,...,5.81,5.98,1.35,1.48,1.28,1.40,77.05,77.61,77.56,78.31


In [445]:
pivoted.columns

Index(['날짜', '난지_평균습도(%)', '서남_평균습도(%)', '중랑_평균습도(%)', '탄천_평균습도(%)',
       '난지_최저습도(%)', '서남_최저습도(%)', '중랑_최저습도(%)', '탄천_최저습도(%)', '난지_최고습도(%)',
       '서남_최고습도(%)', '중랑_최고습도(%)', '탄천_최고습도(%)', '난지_습도표준편차', '서남_습도표준편차',
       '중랑_습도표준편차', '탄천_습도표준편차', '난지_일_평균기온(°C)', '서남_일_평균기온(°C)',
       '중랑_일_평균기온(°C)', '탄천_일_평균기온(°C)', '난지_일_최저기온(°C)', '서남_일_최저기온(°C)',
       '중랑_일_최저기온(°C)', '탄천_일_최저기온(°C)', '난지_일_최고기온(°C)', '서남_일_최고기온(°C)',
       '중랑_일_최고기온(°C)', '탄천_일_최고기온(°C)', '난지_일_최대순간풍속(m/s)',
       '서남_일_최대순간풍속(m/s)', '중랑_일_최대순간풍속(m/s)', '탄천_일_최대순간풍속(m/s)',
       '난지_일_평균풍속(m/s)', '서남_일_평균풍속(m/s)', '중랑_일_평균풍속(m/s)', '탄천_일_평균풍속(m/s)',
       '난지_불쾌지수(DI)', '서남_불쾌지수(DI)', '중랑_불쾌지수(DI)', '탄천_불쾌지수(DI)'],
      dtype='object')

In [446]:
def discomfort_level(di):
    if di < 68:
        return '쾌적'
    elif di < 75:
        return '약간 불쾌'
    elif di < 80:
        return '불쾌'
    elif di < 84:
        return '매우 불쾌'
    else:
        return '극심한 불쾌'

# 불쾌지수 컬럼들 찾기
di_cols = [col for col in pivoted.columns if col.endswith('불쾌지수(DI)')]

# 센터별 등급 컬럼 추가
for col in di_cols:
    center = col.split('_')[0]
    pivoted[f'{center}_불쾌지수등급'] = pivoted[col].apply(discomfort_level)

pivoted

,날짜,난지_평균습도(%),서남_평균습도(%),중랑_평균습도(%),탄천_평균습도(%),난지_최저습도(%),서남_최저습도(%),중랑_최저습도(%),탄천_최저습도(%),난지_최고습도(%),...,중랑_일_평균풍속(m/s),탄천_일_평균풍속(m/s),난지_불쾌지수(DI),서남_불쾌지수(DI),중랑_불쾌지수(DI),탄천_불쾌지수(DI),난지_불쾌지수등급,서남_불쾌지수등급,중랑_불쾌지수등급,탄천_불쾌지수등급
0,2017-01-01,77.11,78.60,80.54,78.89,65.67,65.78,68.67,67.25,87.17,...,0.74,1.05,41.04,41.44,39.91,40.41,쾌적,쾌적,쾌적,쾌적
1,2017-01-02,77.49,77.28,79.31,77.73,61.67,61.50,65.11,61.75,89.00,...,1.47,1.42,44.26,44.93,44.02,44.85,쾌적,쾌적,쾌적,쾌적
2,2017-01-03,63.13,62.31,67.32,61.32,42.17,43.72,43.33,40.75,83.00,...,1.39,1.30,40.93,41.77,40.76,42.53,쾌적,쾌적,쾌적,쾌적
3,2017-01-04,60.61,61.59,63.29,60.07,32.00,28.94,36.44,32.00,81.67,...,1.20,1.18,43.66,44.23,43.70,44.77,쾌적,쾌적,쾌적,쾌적
4,2017-01-05,55.00,54.83,59.72,56.81,43.83,45.28,50.11,45.25,63.17,...,2.06,1.45,44.70,45.15,43.93,44.98,쾌적,쾌적,쾌적,쾌적
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3098,2025-06-26,88.15,84.70,85.41,84.18,77.17,70.22,73.78,72.75,96.83,...,0.99,1.08,69.53,70.51,70.56,70.87,약간 불쾌,약간 불쾌,약간 불쾌,약간 불쾌
3099,2025-06-27,79.09,75.54,79.29,76.58,61.50,58.61,62.44,59.75,91.83,...,0.93,1.08,73.74,74.61,74.05,74.62,약간 불쾌,약간 불쾌,약간 불쾌,약간 불쾌
3100,2025-06-28,85.97,81.58,84.40,80.02,79.00,73.44,76.00,69.75,93.00,...,0.81,1.02,76.13,77.13,76.46,77.47,불쾌,불쾌,불쾌,불쾌
3101,2025-06-29,79.58,76.18,75.91,72.94,62.50,58.33,60.56,58.50,92.50,...,1.28,1.40,77.05,77.61,77.56,78.31,불쾌,불쾌,불쾌,불쾌


In [447]:
pivoted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3103 entries, 0 to 3102
Data columns (total 45 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   날짜                3103 non-null   object 
 1   난지_평균습도(%)        3103 non-null   float64
 2   서남_평균습도(%)        3103 non-null   float64
 3   중랑_평균습도(%)        3103 non-null   float64
 4   탄천_평균습도(%)        3103 non-null   float64
 5   난지_최저습도(%)        3103 non-null   float64
 6   서남_최저습도(%)        3103 non-null   float64
 7   중랑_최저습도(%)        3103 non-null   float64
 8   탄천_최저습도(%)        3103 non-null   float64
 9   난지_최고습도(%)        3103 non-null   float64
 10  서남_최고습도(%)        3103 non-null   float64
 11  중랑_최고습도(%)        3103 non-null   float64
 12  탄천_최고습도(%)        3103 non-null   float64
 13  난지_습도표준편차         3103 non-null   float64
 14  서남_습도표준편차         3103 non-null   float64
 15  중랑_습도표준편차         3103 non-null   float64
 16  탄천_습도표준편차         3103 non-null   float64


In [448]:
df = pd.read_excel(
    '../data/raw/행정구역(동별)_20250807111254.xlsx',
    engine='openpyxl',
    header=2 # 위에 쓸모없는 행 빼고
)

# 병합되어 있는 셀 채우기
df[['동별(1)', '동별(2)', '동별(3)']] = df[['동별(1)', '동별(2)', '동별(3)']].ffill()

# 소계 제외
df_filtered = df[~df['동별(3)'].str.contains("소계")]

# 필요한 컬럼만 선택 및 이름 변경
df_final = df_filtered[['동별(2)', '동별(3)', '면적 (k㎡)']]
df_final.columns = ['행정구', '행정동', '면적_km2']

# 인덱스 리셋
df_final = df_final.reset_index(drop=True)

df_final



c:\Users\user\anaconda3\envs\young\lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,행정구,행정동,면적_km2
0,종로구,사직동,1.23
1,종로구,삼청동,1.49
2,종로구,부암동,2.27
3,종로구,평창동,8.87
4,종로구,무악동,0.36
...,...,...,...
421,강동구,둔촌1동,0.92
422,강동구,둔촌2동,1.56
423,강동구,강일동,2.26
424,강동구,상일1동,1.82


In [449]:
df_final.to_csv('../data/processed/행정동_면적.csv', encoding='utf-8-sig', index=False) # 확인해보니까 종로 1.2.3.4동 이렇게 묶여있는 값들이 있어서 수정 필요

In [450]:
### 강수량 행정동 면적 비율로
rain = pd.read_csv('../data/processed/강수량.csv', encoding='utf-8-sig')
area = pd.read_csv('../data/processed/행정동_면적.csv', encoding='utf-8-sig')
center = pd.read_excel('../data/raw/seoul_mapping_dong_to_center.xlsx', engine='openpyxl')
option = pd.read_excel('../data/raw/seoul_mapping_center_to_districts.xlsx', engine='openpyxl')

# 데이터 확인
print(rain.columns, area.columns, center.columns, option.columns)
print('\n')
print(rain.shape, area.shape, center.shape, option.shape)

Index(['날짜', '행정구', '일_일강수량(mm)'], dtype='object') Index(['행정구', '행정동', '면적_km2'], dtype='object') Index(['자치구', '행정동', '처리센터'], dtype='object') Index(['물재생센터', '관할지역', '전체_일부'], dtype='object')


(74309, 3) (426, 3) (426, 3) (35, 3)


In [451]:
### 행정구 단위의 강수량 데이터를 물재생센터 단위로 변환 ###
## 단, 물재생센터는 전체 행정구를 관할할 수도 있고, 일부 동만 관할할 수도 있으므로,,,
# 전체(전역): 단순 합계
# 일부: 행정동 면적 비율을 곱한 가중 합계
# => 전역은 단순히 합산 일부는 동면적 비율로 강수량 가중치 계산 -> 최종적으로 날짜 × 센터별로 집계

# 관할구역이 전역 / 일부 구분
option_full = option[option['전체_일부'] == '전역']   # 전역: 해당 센터가 행정구 전체를 관할
option_partial = option[option['전체_일부'] == '일부'] # 일부: 해당 센터가 행정구의 일부 동만 관할

# 전역 처리: 행정구 단위 강수량을 센터에 단순 할당
rain_full = rain.merge(option_full, left_on='행정구', right_on='관할지역')
# → '행정구' 단위 강수량 데이터를 전역으로 관할하는 센터에 매핑
rain_full_centered = rain_full.groupby(['날짜', '물재생센터'], as_index=False)['일_일강수량(mm)'].sum()
# → 센터 × 날짜별로 강수량 합산 (전역이므로 단순 합)

# 일부 처리: 행정동 면적 비율로 가중 강수량 계산
#센터별 행정동 관할 정보(center)와 행정동 면적(area) 병합
partial_center = center.merge(area, left_on=['자치구', '행정동'], right_on=['행정구', '행정동'])
# option_partial에 해당하는 자치구(관할 행정구)만 필터링
partial_center = partial_center[partial_center['자치구'].isin(option_partial['관할지역'])]
# 자치구 기준으로 총 면적 합산 후, 각 동의 면적 비율 계산
partial_center['구면적합'] = partial_center.groupby('자치구')['면적_km2'].transform('sum')
partial_center['면적비율'] = partial_center['면적_km2'] / partial_center['구면적합'] # → 예: 성동구에서 센터가 관할하는 동이 차지하는 면적 비율
# 강수량(rain) 데이터와 병합 (행정구 == 자치구 기준)
rain_partial = rain.merge(partial_center, left_on='행정구', right_on='자치구')
# 면적비율을 곱한 가중 강수량 계산
rain_partial['가중강수량'] = rain_partial['일_일강수량(mm)'] * rain_partial['면적비율']
# 센터 × 날짜별로 가중강수량 합산
rain_partial_centered = rain_partial.groupby(['날짜', '처리센터'], as_index=False)['가중강수량'].sum()
# 컬럼 이름 정리
rain_partial_centered = rain_partial_centered.rename(columns={
    '처리센터': '물재생센터',
    '가중강수량': '일_일강수량(mm)'
})

# 전역 + 일부 결과 병합 및 정리
rain_final = pd.concat([rain_full_centered, rain_partial_centered], axis=0)
# → 센터 × 날짜 기준으로 다시 한 번 그룹화 (전역/일부 겹침 방지)
rain_final = rain_final.groupby(['날짜', '물재생센터'], as_index=False)['일_일강수량(mm)'].sum()
rain_final

,날짜,물재생센터,일_일강수량(mm)
0,2017-01-01,난지물재생센터,0.0
1,2017-01-01,서남물재생센터,0.0
2,2017-01-01,중랑물재생센터,0.0
3,2017-01-01,탄천물재생센터,0.0
4,2017-01-02,난지물재생센터,0.0
...,...,...,...
12407,2025-06-29,탄천물재생센터,0.0
12408,2025-06-30,난지물재생센터,0.0
12409,2025-06-30,서남물재생센터,0.0
12410,2025-06-30,중랑물재생센터,0.0


In [452]:
rain_final = rain_final.replace({
    '물재생센터': {
        '탄천물재생센터': '탄천',
        '중랑물재생센터': '중랑',
        '난지물재생센터': '난지',
        '서남물재생센터': '서남'
    }
})

rain_final

,날짜,물재생센터,일_일강수량(mm)
0,2017-01-01,난지,0.0
1,2017-01-01,서남,0.0
2,2017-01-01,중랑,0.0
3,2017-01-01,탄천,0.0
4,2017-01-02,난지,0.0
...,...,...,...
12407,2025-06-29,탄천,0.0
12408,2025-06-30,난지,0.0
12409,2025-06-30,서남,0.0
12410,2025-06-30,중랑,0.0


In [453]:
rain_pivoted = rain_final.pivot(index='날짜', columns='물재생센터', values='일_일강수량(mm)')
rain_pivoted.columns = [f"{center}_일_일강수량(mm)" for center in rain_pivoted.columns]
rain_pivoted = rain_pivoted.reset_index()
# 소수점 첫째 자리까지 반올림
rain_pivoted.iloc[:, 1:] = rain_pivoted.iloc[:, 1:].round(1)
rain_pivoted

,날짜,난지_일_일강수량(mm),서남_일_일강수량(mm),중랑_일_일강수량(mm),탄천_일_일강수량(mm)
0,2017-01-01,0.0,0.0,0.0,0.0
1,2017-01-02,0.0,0.2,1.5,0.0
2,2017-01-03,0.0,0.0,0.0,0.0
3,2017-01-04,0.0,0.0,0.0,0.0
4,2017-01-05,0.0,0.0,0.0,0.0
...,...,...,...,...,...
3098,2025-06-26,4.5,3.9,5.5,2.1
3099,2025-06-27,0.0,0.0,0.0,0.0
3100,2025-06-28,2.5,3.2,6.5,0.5
3101,2025-06-29,0.5,0.0,0.0,0.0


In [454]:
### 강수량까지 데이터 합치기
merged = pivoted.merge(rain_pivoted, on='날짜', how='left')
merged

,날짜,난지_평균습도(%),서남_평균습도(%),중랑_평균습도(%),탄천_평균습도(%),난지_최저습도(%),서남_최저습도(%),중랑_최저습도(%),탄천_최저습도(%),난지_최고습도(%),...,중랑_불쾌지수(DI),탄천_불쾌지수(DI),난지_불쾌지수등급,서남_불쾌지수등급,중랑_불쾌지수등급,탄천_불쾌지수등급,난지_일_일강수량(mm),서남_일_일강수량(mm),중랑_일_일강수량(mm),탄천_일_일강수량(mm)
0,2017-01-01,77.11,78.60,80.54,78.89,65.67,65.78,68.67,67.25,87.17,...,39.91,40.41,쾌적,쾌적,쾌적,쾌적,0.0,0.0,0.0,0.0
1,2017-01-02,77.49,77.28,79.31,77.73,61.67,61.50,65.11,61.75,89.00,...,44.02,44.85,쾌적,쾌적,쾌적,쾌적,0.0,0.2,1.5,0.0
2,2017-01-03,63.13,62.31,67.32,61.32,42.17,43.72,43.33,40.75,83.00,...,40.76,42.53,쾌적,쾌적,쾌적,쾌적,0.0,0.0,0.0,0.0
3,2017-01-04,60.61,61.59,63.29,60.07,32.00,28.94,36.44,32.00,81.67,...,43.70,44.77,쾌적,쾌적,쾌적,쾌적,0.0,0.0,0.0,0.0
4,2017-01-05,55.00,54.83,59.72,56.81,43.83,45.28,50.11,45.25,63.17,...,43.93,44.98,쾌적,쾌적,쾌적,쾌적,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3098,2025-06-26,88.15,84.70,85.41,84.18,77.17,70.22,73.78,72.75,96.83,...,70.56,70.87,약간 불쾌,약간 불쾌,약간 불쾌,약간 불쾌,4.5,3.9,5.5,2.1
3099,2025-06-27,79.09,75.54,79.29,76.58,61.50,58.61,62.44,59.75,91.83,...,74.05,74.62,약간 불쾌,약간 불쾌,약간 불쾌,약간 불쾌,0.0,0.0,0.0,0.0
3100,2025-06-28,85.97,81.58,84.40,80.02,79.00,73.44,76.00,69.75,93.00,...,76.46,77.47,불쾌,불쾌,불쾌,불쾌,2.5,3.2,6.5,0.5
3101,2025-06-29,79.58,76.18,75.91,72.94,62.50,58.33,60.56,58.50,92.50,...,77.56,78.31,불쾌,불쾌,불쾌,불쾌,0.5,0.0,0.0,0.0


In [455]:
total

,날짜,요일,공휴일,난지_목욕장업,난지_세탁업,난지_수영장업,난지_종합체육시설업,난지_체력단련장업,서남_목욕장업,서남_세탁업,...,탄천_2처리장,탄천_합계,중랑_합계,서남_합계,난지_합계,총처리량,난지_하천,서남_하천,중랑_하천,탄천_하천
0,2017-01-01,일요일,1,210,939,28,26,779,455,2253,...,285320,626215.0,1214313.0,1416068.0,492874.0,3749470.0,0.0,0.00,0.00,0.00
1,2017-01-02,월요일,0,210,939,28,26,779,455,2253,...,311049,675134.0,1237077.0,1455896.0,520536.0,3888643.0,0.0,0.67,2.25,0.00
2,2017-01-03,화요일,0,210,939,28,26,779,454,2253,...,301380,674834.0,1224533.0,1466182.0,488454.0,3854003.0,0.0,0.00,0.00,0.00
3,2017-01-04,수요일,0,210,939,28,26,779,454,2253,...,302954,671066.0,1225081.0,1460573.0,510180.0,3866900.0,0.0,0.00,0.00,0.00
4,2017-01-05,목요일,0,210,939,28,26,779,454,2253,...,298534,656065.0,1215632.0,1455815.0,496289.0,3823801.0,0.0,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3098,2025-06-26,목요일,0,120,515,22,20,618,238,1247,...,325456,736100.0,1218638.0,0.0,0.0,1954738.0,7.0,6.00,8.25,5.25
3099,2025-06-27,금요일,0,120,515,22,20,618,238,1247,...,332700,721150.0,1213868.0,0.0,0.0,1935018.0,0.0,0.00,0.00,0.00
3100,2025-06-28,토요일,0,120,515,22,20,618,238,1247,...,321781,717604.0,1232828.0,0.0,0.0,1950432.0,3.5,4.33,12.00,1.50
3101,2025-06-29,일요일,0,120,515,22,20,618,238,1247,...,300016,721513.0,1206254.0,0.0,0.0,1927767.0,0.0,0.00,0.00,0.00


In [456]:
total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3103 entries, 0 to 3102
Data columns (total 49 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   날짜          3103 non-null   datetime64[ns]
 1   요일          3103 non-null   object        
 2   공휴일         3103 non-null   int64         
 3   난지_목욕장업     3103 non-null   int64         
 4   난지_세탁업      3103 non-null   int64         
 5   난지_수영장업     3103 non-null   int64         
 6   난지_종합체육시설업  3103 non-null   int64         
 7   난지_체력단련장업   3103 non-null   int64         
 8   서남_목욕장업     3103 non-null   int64         
 9   서남_세탁업      3103 non-null   int64         
 10  서남_수영장업     3103 non-null   int64         
 11  서남_종합체육시설업  3103 non-null   int64         
 12  서남_체력단련장업   3103 non-null   int64         
 13  중랑_목욕장업     3103 non-null   int64         
 14  중랑_세탁업      3103 non-null   int64         
 15  중랑_수영장업     3103 non-null   int64         
 16  중랑_종합체육시설업  3103 non-nul

In [457]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3103 entries, 0 to 3102
Data columns (total 49 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   날짜                3103 non-null   object 
 1   난지_평균습도(%)        3103 non-null   float64
 2   서남_평균습도(%)        3103 non-null   float64
 3   중랑_평균습도(%)        3103 non-null   float64
 4   탄천_평균습도(%)        3103 non-null   float64
 5   난지_최저습도(%)        3103 non-null   float64
 6   서남_최저습도(%)        3103 non-null   float64
 7   중랑_최저습도(%)        3103 non-null   float64
 8   탄천_최저습도(%)        3103 non-null   float64
 9   난지_최고습도(%)        3103 non-null   float64
 10  서남_최고습도(%)        3103 non-null   float64
 11  중랑_최고습도(%)        3103 non-null   float64
 12  탄천_최고습도(%)        3103 non-null   float64
 13  난지_습도표준편차         3103 non-null   float64
 14  서남_습도표준편차         3103 non-null   float64
 15  중랑_습도표준편차         3103 non-null   float64
 16  탄천_습도표준편차         3103 non-null   float64


In [458]:
merged['날짜'] = pd.to_datetime(merged['날짜'])

In [459]:
### 날짜 컬럼 기준으로 모든 데이터 합치기 ###
# total : (공휴일, 인허가, 인구, 하천), merged : 기상
final = pd.merge(total, merged, on = '날짜', how = 'left')
final

,날짜,요일,공휴일,난지_목욕장업,난지_세탁업,난지_수영장업,난지_종합체육시설업,난지_체력단련장업,서남_목욕장업,서남_세탁업,...,중랑_불쾌지수(DI),탄천_불쾌지수(DI),난지_불쾌지수등급,서남_불쾌지수등급,중랑_불쾌지수등급,탄천_불쾌지수등급,난지_일_일강수량(mm),서남_일_일강수량(mm),중랑_일_일강수량(mm),탄천_일_일강수량(mm)
0,2017-01-01,일요일,1,210,939,28,26,779,455,2253,...,39.91,40.41,쾌적,쾌적,쾌적,쾌적,0.0,0.0,0.0,0.0
1,2017-01-02,월요일,0,210,939,28,26,779,455,2253,...,44.02,44.85,쾌적,쾌적,쾌적,쾌적,0.0,0.2,1.5,0.0
2,2017-01-03,화요일,0,210,939,28,26,779,454,2253,...,40.76,42.53,쾌적,쾌적,쾌적,쾌적,0.0,0.0,0.0,0.0
3,2017-01-04,수요일,0,210,939,28,26,779,454,2253,...,43.70,44.77,쾌적,쾌적,쾌적,쾌적,0.0,0.0,0.0,0.0
4,2017-01-05,목요일,0,210,939,28,26,779,454,2253,...,43.93,44.98,쾌적,쾌적,쾌적,쾌적,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3098,2025-06-26,목요일,0,120,515,22,20,618,238,1247,...,70.56,70.87,약간 불쾌,약간 불쾌,약간 불쾌,약간 불쾌,4.5,3.9,5.5,2.1
3099,2025-06-27,금요일,0,120,515,22,20,618,238,1247,...,74.05,74.62,약간 불쾌,약간 불쾌,약간 불쾌,약간 불쾌,0.0,0.0,0.0,0.0
3100,2025-06-28,토요일,0,120,515,22,20,618,238,1247,...,76.46,77.47,불쾌,불쾌,불쾌,불쾌,2.5,3.2,6.5,0.5
3101,2025-06-29,일요일,0,120,515,22,20,618,238,1247,...,77.56,78.31,불쾌,불쾌,불쾌,불쾌,0.5,0.0,0.0,0.0


In [460]:
final.columns

Index(['날짜', '요일', '공휴일', '난지_목욕장업', '난지_세탁업', '난지_수영장업', '난지_종합체육시설업',
       '난지_체력단련장업', '서남_목욕장업', '서남_세탁업', '서남_수영장업', '서남_종합체육시설업', '서남_체력단련장업',
       '중랑_목욕장업', '중랑_세탁업', '중랑_수영장업', '중랑_종합체육시설업', '중랑_체력단련장업', '탄천_목욕장업',
       '탄천_세탁업', '탄천_수영장업', '탄천_종합체육시설업', '탄천_체력단련장업', '난지_생활인구', '서남_생활인구',
       '중랑_생활인구', '탄천_생활인구', '난지_1처리장', '난지_2처리장', '난지_정화조', '난지_중계펌프장',
       '서남_1처리장', '서남_2처리장', '서남_시설현대화', '중랑_1처리장', '중랑_2처리장', '중랑_3처리장',
       '중랑_4처리장', '탄천_1처리장', '탄천_2처리장', '탄천_합계', '중랑_합계', '서남_합계', '난지_합계',
       '총처리량', '난지_하천', '서남_하천', '중랑_하천', '탄천_하천', '난지_평균습도(%)', '서남_평균습도(%)',
       '중랑_평균습도(%)', '탄천_평균습도(%)', '난지_최저습도(%)', '서남_최저습도(%)', '중랑_최저습도(%)',
       '탄천_최저습도(%)', '난지_최고습도(%)', '서남_최고습도(%)', '중랑_최고습도(%)', '탄천_최고습도(%)',
       '난지_습도표준편차', '서남_습도표준편차', '중랑_습도표준편차', '탄천_습도표준편차', '난지_일_평균기온(°C)',
       '서남_일_평균기온(°C)', '중랑_일_평균기온(°C)', '탄천_일_평균기온(°C)', '난지_일_최저기온(°C)',
       '서남_일_최저기온(°C)', '중랑_일_최저기온(°C)', '탄천_일_최저기온(°C)', '난지_일_최고기온(°C)',
       '서남_일_

In [461]:
final.to_csv('../data/processed/merge_data.csv', encoding='utf-8-sig', index=False)

## 센터별로 나누기

In [462]:
df = pd.read_csv('../data/processed/merge_data.csv', encoding='utf-8-sig')
df

,날짜,요일,공휴일,난지_목욕장업,난지_세탁업,난지_수영장업,난지_종합체육시설업,난지_체력단련장업,서남_목욕장업,서남_세탁업,...,중랑_불쾌지수(DI),탄천_불쾌지수(DI),난지_불쾌지수등급,서남_불쾌지수등급,중랑_불쾌지수등급,탄천_불쾌지수등급,난지_일_일강수량(mm),서남_일_일강수량(mm),중랑_일_일강수량(mm),탄천_일_일강수량(mm)
0,2017-01-01,일요일,1,210,939,28,26,779,455,2253,...,39.91,40.41,쾌적,쾌적,쾌적,쾌적,0.0,0.0,0.0,0.0
1,2017-01-02,월요일,0,210,939,28,26,779,455,2253,...,44.02,44.85,쾌적,쾌적,쾌적,쾌적,0.0,0.2,1.5,0.0
2,2017-01-03,화요일,0,210,939,28,26,779,454,2253,...,40.76,42.53,쾌적,쾌적,쾌적,쾌적,0.0,0.0,0.0,0.0
3,2017-01-04,수요일,0,210,939,28,26,779,454,2253,...,43.70,44.77,쾌적,쾌적,쾌적,쾌적,0.0,0.0,0.0,0.0
4,2017-01-05,목요일,0,210,939,28,26,779,454,2253,...,43.93,44.98,쾌적,쾌적,쾌적,쾌적,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3098,2025-06-26,목요일,0,120,515,22,20,618,238,1247,...,70.56,70.87,약간 불쾌,약간 불쾌,약간 불쾌,약간 불쾌,4.5,3.9,5.5,2.1
3099,2025-06-27,금요일,0,120,515,22,20,618,238,1247,...,74.05,74.62,약간 불쾌,약간 불쾌,약간 불쾌,약간 불쾌,0.0,0.0,0.0,0.0
3100,2025-06-28,토요일,0,120,515,22,20,618,238,1247,...,76.46,77.47,불쾌,불쾌,불쾌,불쾌,2.5,3.2,6.5,0.5
3101,2025-06-29,일요일,0,120,515,22,20,618,238,1247,...,77.56,78.31,불쾌,불쾌,불쾌,불쾌,0.5,0.0,0.0,0.0


In [463]:
df.columns

Index(['날짜', '요일', '공휴일', '난지_목욕장업', '난지_세탁업', '난지_수영장업', '난지_종합체육시설업',
       '난지_체력단련장업', '서남_목욕장업', '서남_세탁업', '서남_수영장업', '서남_종합체육시설업', '서남_체력단련장업',
       '중랑_목욕장업', '중랑_세탁업', '중랑_수영장업', '중랑_종합체육시설업', '중랑_체력단련장업', '탄천_목욕장업',
       '탄천_세탁업', '탄천_수영장업', '탄천_종합체육시설업', '탄천_체력단련장업', '난지_생활인구', '서남_생활인구',
       '중랑_생활인구', '탄천_생활인구', '난지_1처리장', '난지_2처리장', '난지_정화조', '난지_중계펌프장',
       '서남_1처리장', '서남_2처리장', '서남_시설현대화', '중랑_1처리장', '중랑_2처리장', '중랑_3처리장',
       '중랑_4처리장', '탄천_1처리장', '탄천_2처리장', '탄천_합계', '중랑_합계', '서남_합계', '난지_합계',
       '총처리량', '난지_하천', '서남_하천', '중랑_하천', '탄천_하천', '난지_평균습도(%)', '서남_평균습도(%)',
       '중랑_평균습도(%)', '탄천_평균습도(%)', '난지_최저습도(%)', '서남_최저습도(%)', '중랑_최저습도(%)',
       '탄천_최저습도(%)', '난지_최고습도(%)', '서남_최고습도(%)', '중랑_최고습도(%)', '탄천_최고습도(%)',
       '난지_습도표준편차', '서남_습도표준편차', '중랑_습도표준편차', '탄천_습도표준편차', '난지_일_평균기온(°C)',
       '서남_일_평균기온(°C)', '중랑_일_평균기온(°C)', '탄천_일_평균기온(°C)', '난지_일_최저기온(°C)',
       '서남_일_최저기온(°C)', '중랑_일_최저기온(°C)', '탄천_일_최저기온(°C)', '난지_일_최고기온(°C)',
       '서남_일_

In [464]:
### 전처리: 센터별로 데이터 나누기 (불필요한 NaN 제거 포함)
df_long = df.copy()

# 센터 목록
centers = ['난지', '서남', '중랑', '탄천']

# 날짜, 요일, 공휴일은 그대로 유지할 컬럼
id_vars = ['날짜', '요일', '공휴일']

# 센터별 데이터프레임 리스트
dfs = []

for center in centers:
    # 해당 센터 관련 컬럼 추출
    center_cols = [col for col in df_long.columns if col.startswith(center + '_')]

    # 해당 센터 데이터 + 공통 컬럼 복사
    temp = df_long[id_vars + center_cols].copy()

    # long 형식으로 melt
    temp = temp.melt(id_vars=id_vars, var_name='항목', value_name='값')
    temp['항목'] = temp['항목'].str.replace(f'{center}_', '', regex=False)
    temp['센터'] = center

    # wide 형식으로 재구조화
    temp = temp.pivot(index=['날짜', '요일', '공휴일', '센터'], columns='항목', values='값').reset_index()

    # NaN만 있는 열 제거 (해당 센터에 없는 항목 제거)
    temp = temp.dropna(axis=1, how='all')

    # 센터 컬럼 제거
    temp = temp.drop(columns='센터')

    # 리스트에 추가
    dfs.append(temp)

# 병합 완료
df_n_center = dfs[centers.index('난지')]
df_s_center = dfs[centers.index('서남')]
df_g_center = dfs[centers.index('중랑')]
df_t_center = dfs[centers.index('탄천')]


In [465]:
df_n_center

항목,날짜,요일,공휴일,1처리장,2처리장,목욕장업,불쾌지수(DI),불쾌지수등급,생활인구,세탁업,...,일_평균풍속(m/s),정화조,종합체육시설업,중계펌프장,체력단련장업,최고습도(%),최저습도(%),평균습도(%),하천,합계
0,2017-01-01,일요일,1,212474.0,272541.0,210,41.04,쾌적,1920764.58,939,...,1.13,725.0,26,7134,779,87.17,65.67,77.11,0.0,492874.0
1,2017-01-02,월요일,0,224083.0,286338.0,210,44.26,쾌적,2011902.49,939,...,1.65,4147.0,26,5968,779,89.0,61.67,77.49,0.0,520536.0
2,2017-01-03,화요일,0,205711.0,269835.0,210,40.93,쾌적,2021411.02,939,...,1.43,4554.0,26,8354,779,83.0,42.17,63.13,0.0,488454.0
3,2017-01-04,수요일,0,219468.0,277305.0,210,43.66,쾌적,2016146.44,939,...,1.27,5100.0,26,8307,779,81.67,32.0,60.61,0.0,510180.0
4,2017-01-05,목요일,0,207189.0,275215.0,210,44.7,쾌적,2019225.84,939,...,2.43,5733.0,26,8152,779,63.17,43.83,55.0,0.0,496289.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3098,2025-06-26,목요일,0,0.0,0.0,120,69.53,약간 불쾌,1820483.24,515,...,1.02,0.0,20,0,618,96.83,77.17,88.15,7.0,0.0
3099,2025-06-27,금요일,0,0.0,0.0,120,73.74,약간 불쾌,1818477.77,515,...,0.97,0.0,20,0,618,91.83,61.5,79.09,0.0,0.0
3100,2025-06-28,토요일,0,0.0,0.0,120,76.13,불쾌,1751408.17,515,...,0.9,0.0,20,0,618,93.0,79.0,85.97,3.5,0.0
3101,2025-06-29,일요일,0,0.0,0.0,120,77.05,불쾌,1716949.92,515,...,1.35,0.0,20,0,618,92.5,62.5,79.58,0.0,0.0


In [466]:
print(df_s_center.columns)
print(df_g_center.columns)
print(df_t_center.columns)
print(df_n_center.columns)

Index(['날짜', '요일', '공휴일', '1처리장', '2처리장', '목욕장업', '불쾌지수(DI)', '불쾌지수등급', '생활인구',
       '세탁업', '수영장업', '습도표준편차', '시설현대화', '일_일강수량(mm)', '일_최고기온(°C)',
       '일_최대순간풍속(m/s)', '일_최저기온(°C)', '일_평균기온(°C)', '일_평균풍속(m/s)', '종합체육시설업',
       '체력단련장업', '최고습도(%)', '최저습도(%)', '평균습도(%)', '하천', '합계'],
      dtype='object', name='항목')
Index(['날짜', '요일', '공휴일', '1처리장', '2처리장', '3처리장', '4처리장', '목욕장업', '불쾌지수(DI)',
       '불쾌지수등급', '생활인구', '세탁업', '수영장업', '습도표준편차', '일_일강수량(mm)', '일_최고기온(°C)',
       '일_최대순간풍속(m/s)', '일_최저기온(°C)', '일_평균기온(°C)', '일_평균풍속(m/s)', '종합체육시설업',
       '체력단련장업', '최고습도(%)', '최저습도(%)', '평균습도(%)', '하천', '합계'],
      dtype='object', name='항목')
Index(['날짜', '요일', '공휴일', '1처리장', '2처리장', '목욕장업', '불쾌지수(DI)', '불쾌지수등급', '생활인구',
       '세탁업', '수영장업', '습도표준편차', '일_일강수량(mm)', '일_최고기온(°C)', '일_최대순간풍속(m/s)',
       '일_최저기온(°C)', '일_평균기온(°C)', '일_평균풍속(m/s)', '종합체육시설업', '체력단련장업',
       '최고습도(%)', '최저습도(%)', '평균습도(%)', '하천', '합계'],
      dtype='object', name='항목')
Index(['날짜', '요일', '공휴일', '1처리장',

In [467]:
### 컬럼 보기 좋게 순서 변경 ###
# 보기 좋은 컬럼 순서 정의 함수
# 데이터마다 다른 컬럼(예: 3처리장, 4처리장, 정화조, 중계펌프장, 시설현대화 등)을 있으면 그 순서로 배치, 없으면 자동으로 건너뜀 처리
def desired_column_order():
    # 기본 키/캘린더
    base = ["날짜", "요일", "공휴일"]
    # 업종(인허가)
    biz = ["목욕장업", "세탁업", "수영장업", "종합체육시설업", "체력단련장업"]
    # 하천 집계
    river = ["하천"]
    # 인구/지수
    people_index = ["생활인구", "불쾌지수(DI)", "불쾌지수등급"]
    # 기상 (강수 → 기온(최저/평균/최고) → 풍속 → 습도(최저/평균/최고) → 편차)
    weather = [
        "일_일강수량(mm)",
        "일_최저기온(°C)", "일_평균기온(°C)", "일_최고기온(°C)",
        "일_평균풍속(m/s)", "일_최대순간풍속(m/s)",
        "최저습도(%)", "평균습도(%)", "최고습도(%)",
        "습도표준편차",
    ]
    # 시설/처리장 계열
    plants = ["1처리장", "2처리장", "3처리장", "4처리장", "정화조", "중계펌프장", "시설현대화"]
    # 총합
    tail = ["합계"]
    return base + biz + river + people_index + weather + plants + tail

# 컬럼 재정렬 유틸
# - 지정 순서에 있는 컬럼만 먼저 배치
# - 지정하지 않은(잡다한) 컬럼은 유지하되 맨 끝에 보냄
def reorder_columns(df: pd.DataFrame, keep_unspecified: bool = True) -> pd.DataFrame:
    order = desired_column_order()
    # 실제 존재하는 컬럼만 추림(교집합)
    ordered_exist = [c for c in order if c in df.columns]
    if keep_unspecified:
        # 지정 순서에 없지만 DF에 남아있는 컬럼들(원래 순서 유지)
        rest = [c for c in df.columns if c not in ordered_exist]
        new_cols = ordered_exist + rest
    else:
        new_cols = ordered_exist
    return df.loc[:, new_cols]


In [468]:
# 각 센터에 적용
df_n_center = reorder_columns(df_n_center, keep_unspecified=True)
df_s_center = reorder_columns(df_s_center, keep_unspecified=True)
df_t_center = reorder_columns(df_t_center, keep_unspecified=True)
df_g_center = reorder_columns(df_g_center, keep_unspecified=True)

# 확인: 앞쪽 몇 개 컬럼과 전체 개수
for name, d in {
    "난지(df_n_center)": df_n_center,
    "서남(df_s_center)": df_s_center,
    "탄천(df_t_center)": df_t_center,
    "중랑(df_g_center)": df_g_center,
}.items():
    print(f"\n[{name}] shape={d.shape}")
    print(d.columns.tolist()) 



[난지(df_n_center)] shape=(3103, 27)
['날짜', '요일', '공휴일', '목욕장업', '세탁업', '수영장업', '종합체육시설업', '체력단련장업', '하천', '생활인구', '불쾌지수(DI)', '불쾌지수등급', '일_일강수량(mm)', '일_최저기온(°C)', '일_평균기온(°C)', '일_최고기온(°C)', '일_평균풍속(m/s)', '일_최대순간풍속(m/s)', '최저습도(%)', '평균습도(%)', '최고습도(%)', '습도표준편차', '1처리장', '2처리장', '정화조', '중계펌프장', '합계']

[서남(df_s_center)] shape=(3103, 26)
['날짜', '요일', '공휴일', '목욕장업', '세탁업', '수영장업', '종합체육시설업', '체력단련장업', '하천', '생활인구', '불쾌지수(DI)', '불쾌지수등급', '일_일강수량(mm)', '일_최저기온(°C)', '일_평균기온(°C)', '일_최고기온(°C)', '일_평균풍속(m/s)', '일_최대순간풍속(m/s)', '최저습도(%)', '평균습도(%)', '최고습도(%)', '습도표준편차', '1처리장', '2처리장', '시설현대화', '합계']

[탄천(df_t_center)] shape=(3103, 25)
['날짜', '요일', '공휴일', '목욕장업', '세탁업', '수영장업', '종합체육시설업', '체력단련장업', '하천', '생활인구', '불쾌지수(DI)', '불쾌지수등급', '일_일강수량(mm)', '일_최저기온(°C)', '일_평균기온(°C)', '일_최고기온(°C)', '일_평균풍속(m/s)', '일_최대순간풍속(m/s)', '최저습도(%)', '평균습도(%)', '최고습도(%)', '습도표준편차', '1처리장', '2처리장', '합계']

[중랑(df_g_center)] shape=(3103, 27)
['날짜', '요일', '공휴일', '목욕장업', '세탁업', '수영장업', '종합체육시설업', '체력단련장업', '하천', '생

In [469]:
df_s_center.to_csv('../data/processed/서남.csv', index=False, encoding='utf-8-sig')
df_g_center.to_csv('../data/processed/중랑.csv', index=False, encoding='utf-8-sig')
df_t_center.to_csv('../data/processed/탄천.csv', index=False, encoding='utf-8-sig')
df_n_center.to_csv('../data/processed/난지.csv', index=False, encoding='utf-8-sig')

print(df_s_center.columns)
print(df_g_center.columns)
print(df_t_center.columns)
print(df_n_center.columns)

print(df_s_center.shape)
print(df_g_center.shape)
print(df_t_center.shape)
print(df_n_center.shape)


Index(['날짜', '요일', '공휴일', '목욕장업', '세탁업', '수영장업', '종합체육시설업', '체력단련장업', '하천',
       '생활인구', '불쾌지수(DI)', '불쾌지수등급', '일_일강수량(mm)', '일_최저기온(°C)', '일_평균기온(°C)',
       '일_최고기온(°C)', '일_평균풍속(m/s)', '일_최대순간풍속(m/s)', '최저습도(%)', '평균습도(%)',
       '최고습도(%)', '습도표준편차', '1처리장', '2처리장', '시설현대화', '합계'],
      dtype='object', name='항목')
Index(['날짜', '요일', '공휴일', '목욕장업', '세탁업', '수영장업', '종합체육시설업', '체력단련장업', '하천',
       '생활인구', '불쾌지수(DI)', '불쾌지수등급', '일_일강수량(mm)', '일_최저기온(°C)', '일_평균기온(°C)',
       '일_최고기온(°C)', '일_평균풍속(m/s)', '일_최대순간풍속(m/s)', '최저습도(%)', '평균습도(%)',
       '최고습도(%)', '습도표준편차', '1처리장', '2처리장', '3처리장', '4처리장', '합계'],
      dtype='object', name='항목')
Index(['날짜', '요일', '공휴일', '목욕장업', '세탁업', '수영장업', '종합체육시설업', '체력단련장업', '하천',
       '생활인구', '불쾌지수(DI)', '불쾌지수등급', '일_일강수량(mm)', '일_최저기온(°C)', '일_평균기온(°C)',
       '일_최고기온(°C)', '일_평균풍속(m/s)', '일_최대순간풍속(m/s)', '최저습도(%)', '평균습도(%)',
       '최고습도(%)', '습도표준편차', '1처리장', '2처리장', '합계'],
      dtype='object', name='항목')
Index(['날짜', '요일', '공휴일', '목욕장업',

### 센터별, 계절별로 나누기 ###

In [470]:
### 봄: 3,4,5, 여름: 6,7,8, 가을: 9,10,11, 겨울: 12,1,2 ###
def add_season_column(df: pd.DataFrame, date_col: str = "날짜", season_col: str = "계절"):
    df = df.copy()
    # 날짜가 문자열이면 datetime으로 변환 (이미 datetime이면 그대로 동작)
    df[date_col] = pd.to_datetime(df[date_col])
    month = df[date_col].dt.month

    # 월→계절 매핑
    season_map = {
        12: "겨울", 1: "겨울", 2: "겨울",
        3: "봄", 4: "봄", 5: "봄",
        6: "여름", 7: "여름", 8: "여름",
        9: "가을", 10: "가을", 11: "가을",
    }
    df[season_col] = month.map(season_map)

    # 보기 좋게 카테고리 순서 지정
    cat_order = ["봄", "여름", "가을", "겨울"]
    df[season_col] = pd.Categorical(df[season_col], categories=cat_order, ordered=True)
    return df

In [471]:
# 하나의 센터 DF를 계절별로 잘라 dict로 반환
# 반환 예: {"난지_봄": DataFrame, "난지_여름": DataFrame, ...}
def split_by_season(df: pd.DataFrame, center_kor_name: str,
                    season_col: str = "계절"):
    df = df.copy()
    season_keys = ["봄", "여름", "가을", "겨울"]
    out = {}
    for s in season_keys:
        key = f"{center_kor_name}_{s}"
        out[key] = df[df[season_col] == s].copy()
    return out


In [472]:
# 입력: 이미 만들어진 센터별 DF에 '계절' 컬럼 부여 후 계절단위로 쪼개기

# 계절 컬럼 추가
df_n_season = add_season_column(df_n_center)
df_s_season = add_season_column(df_s_center)
df_t_season = add_season_column(df_t_center)
df_g_season = add_season_column(df_g_center)

# 계절별 분할
split_dict = {}
split_dict.update(split_by_season(df_n_season, "난지"))
split_dict.update(split_by_season(df_s_season, "서남"))
split_dict.update(split_by_season(df_t_season, "탄천"))
split_dict.update(split_by_season(df_g_season, "중랑"))

# split_dict 구성
# ['난지_봄','난지_여름','난지_가을','난지_겨울',
#  '서남_봄','서남_여름','서남_가을','서남_겨울',
#  '탄천_봄','탄천_여름','탄천_가을','탄천_겨울',
#  '중랑_봄','중랑_여름','중랑_가을','중랑_겨울']
# 난지 봄 데이터 확인 : split_dict['난지_봄'].head()


In [473]:
# ['난지_봄','난지_여름','난지_가을','난지_겨울',
#  '서남_봄','서남_여름','서남_가을','서남_겨울',
#  '탄천_봄','탄천_여름','탄천_가을','탄천_겨울',
#  '중랑_봄','중랑_여름','중랑_가을','중랑_겨울']

### 난지 계절별 데이터 저장 ### 
split_dict['난지_봄'].to_csv('../data/processed/center_season/nanji/난지_봄.csv', index=False, encoding='utf-8-sig')
split_dict['난지_여름'].to_csv('../data/processed/center_season/nanji/난지_여름.csv', index=False, encoding='utf-8-sig')
split_dict['난지_가을'].to_csv('../data/processed/center_season/nanji/난지_가을.csv', index=False, encoding='utf-8-sig')
split_dict['난지_겨울'].to_csv('../data/processed/center_season/nanji/난지_겨울.csv', index=False, encoding='utf-8-sig')

### 서남 계절별 데이터 저장 ### 
split_dict['서남_봄'].to_csv('../data/processed/center_season/seonam/서남_봄.csv', index=False, encoding='utf-8-sig')
split_dict['서남_여름'].to_csv('../data/processed/center_season/seonam/서남_여름.csv', index=False, encoding='utf-8-sig')
split_dict['서남_가을'].to_csv('../data/processed/center_season/seonam/서남_가을.csv', index=False, encoding='utf-8-sig')
split_dict['서남_겨울'].to_csv('../data/processed/center_season/seonam/서남_겨울.csv', index=False, encoding='utf-8-sig')

### 탄천 계절별 데이터 저장 ### 
split_dict['탄천_봄'].to_csv('../data/processed/center_season/tancheon/탄천_봄.csv', index=False, encoding='utf-8-sig')
split_dict['탄천_여름'].to_csv('../data/processed/center_season/tancheon/탄천_여름.csv', index=False, encoding='utf-8-sig')
split_dict['탄천_가을'].to_csv('../data/processed/center_season/tancheon/탄천_가을.csv', index=False, encoding='utf-8-sig')
split_dict['탄천_겨울'].to_csv('../data/processed/center_season/tancheon/탄천_겨울.csv', index=False, encoding='utf-8-sig')

### 중랑 계절별 데이터 저장 ### 
split_dict['중랑_봄'].to_csv('../data/processed/center_season/jungnang/중랑_봄.csv', index=False, encoding='utf-8-sig')
split_dict['중랑_여름'].to_csv('../data/processed/center_season/jungnang/중랑_여름.csv', index=False, encoding='utf-8-sig')
split_dict['중랑_가을'].to_csv('../data/processed/center_season/jungnang/중랑_가을.csv', index=False, encoding='utf-8-sig')
split_dict['중랑_겨울'].to_csv('../data/processed/center_season/jungnang/중랑_겨울.csv', index=False, encoding='utf-8-sig')

In [474]:
# 집계 예시 : 연도×계절 축으로 보고 싶을 때(연도별 계절 합계 피벗)
# - 출력 컬럼을 '난지_봄', '난지_여름', ... 식으로 맞추고,
#   index는 연도로 두는 형태

def yearly_season_pivot(df: pd.DataFrame, center_name: str) -> pd.DataFrame:
    tmp = df.copy()
    tmp["연도"] = tmp["날짜"].dt.year
    # 연도×계절별 합계
    p = tmp.groupby(["연도", "계절"], observed=True)["합계"].sum().unstack("계절")
    # 열 이름을 '센터_계절'로 변경
    p.columns = [f"{center_name}_{c}" for c in p.columns]
    return p

p_n = yearly_season_pivot(df_n_season, "난지")
p_s = yearly_season_pivot(df_s_season, "서남")
p_t = yearly_season_pivot(df_t_season, "탄천")
p_g = yearly_season_pivot(df_g_season, "중랑")

# 연도 기준으로 합치기(outer join)
yearly_season_summary = (
    p_n.join(p_s, how="outer")
      .join(p_t, how="outer")
      .join(p_g, how="outer")
      .sort_index()
)

yearly_season_summary.head()


,난지_봄,난지_여름,난지_가을,난지_겨울,서남_봄,서남_여름,서남_가을,서남_겨울,탄천_봄,탄천_여름,탄천_가을,탄천_겨울,중랑_봄,중랑_여름,중랑_가을,중랑_겨울
연도,,,,,,,,,,,,,,,,
2017,48450584.0,67895612.0,50485156.76,46040644.0,137490254.0,165174735.0,140898890.0,134659382.0,66249137.0,79838311.0,67049819.0,63883779.0,108107004.0,145568545.0,115304772.0,109240546.0
2018,59224120.9,59240292.82,54154524.29,48115301.16,150331610.0,154768325.0,145626456.0,134634012.0,76345495.0,74467252.0,66570699.0,63772042.0,122534582.0,127181383.0,119938362.0,105376644.0
2019,49408788.44,56883964.71,54319303.69,46522894.43,137090797.0,153965977.0,146930553.0,131500882.0,61086175.0,67896440.0,65459431.0,59067788.0,103744813.0,114569665.0,111365974.0,100911407.0
2020,48135340.0,63467292.0,54515589.6,48302998.6,137059860.0,162221182.0,153211146.0,135699670.0,63636241.5,75803970.0,72599050.0,63333601.0,110563205.47,130355891.0,119014974.0,110245980.82
2021,57378257.0,57619180.0,55586754.0,48207026.0,157583048.0,159302163.0,150049888.3,136579790.0,69315363.0,72743776.0,69542717.0,62168145.0,117800918.0,116911343.0,113472585.0,108864746.0
